In [1]:
import pandas as pd

df= pd.read_csv('coba1.tsv', sep='\t', header=None)

In [2]:
df.head()

,0,1
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,lokasi strategis di jalan sumatera bandung . t...,positive
2,betapa bahagia nya diri ini saat unboxing pake...,positive
3,"makanan beragam , harga makanan di food stall ...",positive
4,"tempat unik , bagus buat foto , makanan enak ,...",positive


In [3]:
df.shape

(3348, 2)

In [4]:
df.columns =['Text', 'Class']
df

,Text,Class
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,lokasi strategis di jalan sumatera bandung . t...,positive
2,betapa bahagia nya diri ini saat unboxing pake...,positive
3,"makanan beragam , harga makanan di food stall ...",positive
4,"tempat unik , bagus buat foto , makanan enak ,...",positive
...,...,...
3343,dapat bonus tcash 12.500 tapi tidak bisa dipak...,negative
3344,ketahuan yang bawain berita ini goblok banget ...,negative
3345,tidak berfaedah,negative
3346,itu ibu-ibu datang kerumah gue setiap tanggal1...,negative


In [5]:
df.Class.value_counts()

neutral     1148
positive    1100
negative    1100
Name: Class, dtype: int64

In [6]:
import re 

def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('user',' ',text) # Remove every username
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text
    
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

#import dictionary alay untuk mapping kata2 tidak baku  
alay_dict = pd.read_csv('new_kamusalay.csv', encoding='latin-1', header=None)
#rename kolum pada dictionary 
alay_dict = alay_dict.rename(columns={0:'original',
                                    1:'replacement'})

#mengubah kata-kata yang tidak baku pada teks 
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def baku(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

#merubah kata kata alay menjadi kata kata baku
#def normalize_alay(text):
    #return ' '.join([data_alay_new_map[word] if word in data_alay_new_map else word for word in text.split(' ')])
def preprocess(text):
    text = lowercase(text) # 1
    text = remove_nonaplhanumeric(text) # 2
    text = remove_unnecessary_char(text) # 2
    text = baku(text) # 3
    return text


In [7]:
df['text_clean'] = df.Text.apply(preprocess)

In [8]:
df.head()

,Text,Class,text_clean
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini dimiliki oleh pengusaha pabrik tahu...
1,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis di jalan sumatra bandung temp...
2,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia nya diri ini saat unboxing pake...
3,"makanan beragam , harga makanan di food stall ...",positive,makanan beragam harga makanan di food stall ak...
4,"tempat unik , bagus buat foto , makanan enak ,...",positive,tempat unik bagus buat foto makanan enak pegaw...


In [9]:

data_preprocessed = df.text_clean.tolist()

In [10]:
data_preprocessed

['warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung tahu berkualitas dipadu keahlian memasak dipadu kretivitas jadilah warung yang menyajikan menu utama berbahan tahu ditambah menu umum lain sepe i ayam semuanya selera indonesia harga cukup terjangkau jangan melewati tahu bletoka nya tidak kalah dengan yang asli dari tegal ',
 'lokasi strategis di jalan sumatra bandung tempat nya nyaman terutama sofa di lantai 2 paella nya enak sangat pas dimakan dengan minum bir dingin appetiser nya juga enak enak ',
 'betapa bahagia nya diri ini saat unboxing paket dan barang nya bagus menetapkan beli lagi ',
 'makanan beragam harga makanan di food stall akan ditambahkan 10 lagi di kasir suasana ramai dan perlu perhatian untuk mendapatkan parkir dan tempat duduk ',
 'tempat unik bagus buat foto makanan enak pegawai ramah bersih dan luas wifi kencang harga standar dan sesuai dengan tempat nya ada menu masakan makanan barat dan indonesia menu

In [11]:

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(data_preprocessed)

X = tfidf_vect.transform(data_preprocessed)
print ("Feature Extraction selesai")


Feature Extraction selesai


In [13]:
import pickle

pickle.dump(tfidf_vect, open("feature.pickle", "wb"))

In [14]:
from sklearn.model_selection import train_test_split

classes = df.Class

In [15]:
classes

0       positive
1       positive
2       positive
3       positive
4       positive
          ...   
3343    negative
3344    negative
3345    negative
3346    negative
3347    negative
Name: Class, Length: 3348, dtype: object

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size = 0.2)

In [17]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier() 
model.fit(X_train, y_train)

print ("Training selesai")

Training selesai


In [18]:
pickle.dump(model, open("model.pickle", "wb"))

In [19]:
from sklearn.metrics import classification_report

test = model.predict(X_test)

print ("Testing selesai")

print(classification_report(y_test, test)) 

Testing selesai
              precision    recall  f1-score   support

    negative       0.80      0.78      0.79       226
     neutral       0.83      0.86      0.85       221
    positive       0.84      0.83      0.83       223

    accuracy                           0.82       670
   macro avg       0.82      0.82      0.82       670
weighted avg       0.82      0.82      0.82       670



In [20]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=5,random_state=42,shuffle=True)

accuracies = []

y = classes

for iteration, data in enumerate(kf.split(X), start=1):

    data_train   = X[data[0]]
    target_train = y[data[0]]

    data_test    = X[data[1]]
    target_test  = y[data[1]]

    clf = MLPClassifier()
    clf.fit(data_train,target_train)

    preds = clf.predict(data_test)

    # for the current fold only    
    accuracy = accuracy_score(target_test,preds)

    print("Training ke-", iteration)
    print(classification_report(target_test,preds))
    print("======================================================")

    accuracies.append(accuracy)

# this is the average accuracy over all folds
average_accuracy = np.mean(accuracies)

print()
print()
print()
print("Rata-rata Accuracy: ", average_accuracy)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.75      0.82      0.78       200
     neutral       0.85      0.87      0.86       221
    positive       0.90      0.81      0.85       249

    accuracy                           0.83       670
   macro avg       0.83      0.83      0.83       670
weighted avg       0.84      0.83      0.84       670

Training ke- 2
              precision    recall  f1-score   support

    negative       0.75      0.79      0.77       226
     neutral       0.90      0.82      0.86       242
    positive       0.79      0.83      0.81       202

    accuracy                           0.81       670
   macro avg       0.81      0.81      0.81       670
weighted avg       0.82      0.81      0.81       670

Training ke- 3
              precision    recall  f1-score   support

    negative       0.83      0.77      0.80       229
     neutral       0.87      0.89      0.88       227
    positive       0.84      0

In [21]:
original_text =  '''
setiap bumn dibentuk dengan uu bukan dibentuk dengan uu
'''

# Feature Extraction
text = tfidf_vect.transform([preprocess(original_text)])

# Kita prediksi sentimennya
result = model.predict(text)[0]
print("Sentiment:")
print()
print(result)

Sentiment:

neutral
